# Исследование надежности заемщиков.

Исследование данных в финансовой сфере.
Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

## Шаг 1. Обзор данных

### Просмотр данных

In [1]:
import pandas as pd
try:
    df = pd.read_csv(r'C:\Users\Admin\Desktop\Обучение\sprint_2\data.csv')
except: 
    df = pd.read_csv('/datasets/data.csv')

In [2]:
#узнаем общую информацию о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
#визуально просмотрим данные
display(df.head(10)) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В предоставленных данных, на первый взгляд видные следующие аномалии:
    - Пропуски в столбцах 'days_employed' и 'total_income'
    - В столбце 'days_employed' отрицательный стаж
    - В столбце 'days_employed' значения стажа больше прожитых лет
    - В столбце 'days_employed' значения в виде вещественных чисел
    - В столбце 'education' имеются данные с разным регистром
    - В столбце 'gender' используется верхний регистр
    - В столбце 'total_income' имеются значения с большим количеством знаков после запятой

## Шаг 2 Предобработка данных

### Обработка пропусков

In [4]:
#найдем пропуски
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
#выведем пустые значения на экран
df[df['total_income'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Чтобы изучить ошибки в стобцах отлично подойдет метод value_counts() для каждого столбца.

In [6]:
#посмотрим уникальные значения столбца
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

 - df['children'] видим следующие аномалии:
  - количество детей -1, отрицательное количество детей невозможно, скорректируем на положительное число
  - количество детей 20, число слишком велико, предположим, что это опечатка, т.к. 0 находится рядом с цифрой 2

In [7]:
#посмотрим уникальные значения столбца
df['days_employed'].value_counts()

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

 - df['days_employed'] следующие аномалии:
  - отрицательный стаж
  - большое число трудового стажа

In [8]:
#посмотрим уникальные значения столбца
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

 - df['dob_years'] видим следующие аномалии:
  - значения в возрасте "0" и "2"; 0 - скорей всего клиент не сообщил о своем возрасте
  - 2 - опечатка

In [9]:
#посмотрим уникальные значения столбца
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

 - df['education'] видим следующие аномалии:
  - разный регитср в значениях
  - значени "2", предположительно, закончено два класса или два высших

In [10]:
#посмотрим уникальные значения столбца
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

 - df['education_id'] без аномалий

In [11]:
#посмотрим уникальные значения столбца
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

 - df['family_status'] без аномалий

In [12]:
#посмотрим уникальные значения столбца
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

 - df['family_status_id'] без аномалий

In [13]:
#посмотрим уникальные значения столбца
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

 - df['gender'] видим следующие аномалии:
  - регистр значений верхний
  - имеется значение XNA

In [14]:
#посмотрим уникальные значения столбца
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

 - df['income_type'] без аномалий

In [15]:
#посмотрим уникальные значения столбца
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

 - df['debt'] без аномалий

In [16]:
#посмотрим уникальные значения столбца
df['total_income'].value_counts()

169846.427535    1
257737.077768    1
200508.675866    1
106196.235958    1
248730.171354    1
                ..
175057.266090    1
101516.604975    1
239154.168013    1
165009.733021    1
189255.286637    1
Name: total_income, Length: 19351, dtype: int64

 - df['total_income'] без аномалий

In [17]:
#посмотрим уникальные значения столбца
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

 - df['purpose'] имеются неявные дубликаты
 - опечатка "ремонт жилью

### Обработка ошибок в данных

In [18]:
# исправим значение -1 для количества детей согласно нашей гипотезы (т.е. заменим на 1)
df.loc[df['children'] == -1, 'children'] = 1
# проверим
df.loc[df['children'] == -1]['children'].count()

0

In [19]:
# исправим значения 20 для возраста согласно нашей гипотезе
df.loc[df['children'] == 20, 'children'] = 2
# проверим
df.loc[df['children'] == 20]['children'].count()

0

In [20]:
#приведем к нижнему регистру
df['education'] = df['education'].str.lower()

In [21]:
#приведем к нижнему регистру
df['gender'] = df['gender'].str.lower()
# исправим значение пола
df.loc[df['gender'] == 'xna', 'gender'] = 'f'
# проверим
df.loc[df['gender'] == 'xna']['gender'].count()

0

In [22]:
# исправим опечатку в purpose
df.loc[df['purpose'] == 'ремонт жилью', 'purpose'] = 'ремонт жилья'
# проверим
df.loc[df['purpose'] == 'ремонт жилью']['purpose'].count()

0

In [23]:
#проверим долю пропущенных значений
print(f"Доля пропущенных значений составлят:{(df['total_income'].isna().sum()) / len(df['total_income']):.0%}")

Доля пропущенных значений составлят:10%


Видим, что доля пропусков составляет 10%, что может сказаться на анализе. Необходимо заполнить пропуски. Заполним их медианным значением чтобы не учитывать "всплески".

Возможные причины могут быть следующими:
    - пропуск в связи с отсутсвием трудового стажа
    - пропуск т.к. в данный момент клиент безработный

Имеются пропуски в двух столбцах. Столбец 'days_employed' трогать не будем, т.к. он в исследовании не пригодится

In [24]:
#В случае если бы данные в столбце 'days_employed' нам были бы нужны, тогда необходимо было бы их привести к рельным данным.

#Сперва необходимо устранить отрицательные значения, взяв модуль числа (abs())
df['days_employed'] = abs(df['days_employed'])

#Сменим тип из вещественного числа на целые, найдем медиану
days_employed_median = int(df['days_employed'].median())
display(days_employed_median)

#Заменим пропуски
df['days_employed'] = df['days_employed'].fillna(days_employed_median)

#Проверим
display(df['days_employed'].isna().sum())


2194

0

Вернемся к удалению пропусков в столбце 'total_income'

In [25]:
#найдем медиану значений
total_income_median = df['total_income'].median()
display(total_income_median)

145017.93753253992

Так как данные имеют большое количество после запятой, приведем их к целому числу, воспользовавшись методом astype()

In [26]:
#заменим пропуски
df['total_income'] = df['total_income'].fillna(total_income_median).astype('int')

In [27]:
#проверим
display(df['total_income'].isna().sum())

0

### Вывод

В предоставленных данных имелись пропуски в двух столбцах.
Для анализа данных в соответствии с поставленной задачей, нужно было восставновить пропуски в столбце 'total_income'.
Пропуски восстанавливали путем нахождения медианы значений в данном столбце. Обоснование: т.к. в данных имеются значения отличающихся друг от друга на порядки, тем самым вызывая "всплески", было решено найти медиану.
Причины появления пропусков могут быть следующими: отсутсвие рабочего стажа, отсутвие официального рабочего стажа, ошибка при внесении данных.
Сообщить ответсвенному лицу о аномалиях в данных.

## Обработка дубликатов

In [28]:
#проверим количество явных дубликатов
df.duplicated().sum()

71

In [29]:
#удалим дубликаты
df = df.drop_duplicates().reset_index(drop=True)
#проверим
df.duplicated().sum()

0

Явных дубликатов было не так много. Скорей всего были задублированы.

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [31]:
#формируем новый датафрейм - "словарь"
education_data = df[['education_id', 'education']]
# удалим дубликаты из словаря
education_data = education_data.drop_duplicates().reset_index(drop=True)
# проверим содержимое
education_data.head(10)


,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [32]:
#формируем новый датафрейм - "словарь"
family_status_data = df[['family_status_id', 'family_status']]
# удалим дубликаты из словаря
family_status_data = family_status_data.drop_duplicates().reset_index(drop=True)
# проверим содержимое
family_status_data.head(10)


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [33]:
#удалим столбцы education и family_status
df.drop(['education', 'family_status'], axis=1)
#проверим
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int64  
 11  purpose           21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int64  
 11  purpose           21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Создание категорий по столбцу 'total_income'

In [35]:
#создаем функцию, которая пробежится по столбцы 'total_income' и каждое значение в толбце отнесет к категории.
#Если:0–30000 — 'E'; 30001–50000 — 'D'; 50001–200000 — 'C'; 200001–1000000 — 'B'; 1000001 и выше — 'A'.

def func_income_category(total_income):
    if total_income < 30000:
        return 'E'
    if 30001 <= total_income < 50000:
        return 'D'
    if 50001 <= total_income < 200000:
        return 'C'
    if 200001 <= total_income < 1000000:
        return 'B'
    return 'A'


df['total_income_category'] = df['total_income'].apply(func_income_category)

# проверим
df['total_income_category'].value_counts()

C    16016
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Создание категорий по столбцу 'purpose'

In [36]:
#создаем функцию, которая пробежится по столбцу 'purpose' и в зависимости от корня слова определяющего нужду кредита,
#присвоем ей одну из категорий: ‘операции с автомобилем’, ‘операции с недвижимостью’, ‘проведение свадьбы’, ‘получение образования’.

def func_purpose_category(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жил' in purpose:
        return 'операции с недвижимостью'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'свад' in purpose:
        return 'проведение свадьбы'
    if 'образ' in purpose:
        return 'получение образования'
df['purpose_category'] = df['purpose'].apply(func_purpose_category) 

#проверим
df['purpose_category'].value_counts()

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

### Ответы на вопросы.

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [37]:
# функция, что посчитает нам отношение в процентах, будем использовать для ответов на все вопросы
def make_proportion(pdSerises):
    return str(round((pdSerises.sum() / pdSerises.count()) * 100, 2)) + '%'

# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['children'], values=['debt'], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot

,sum,count,make_proportion
,debt,debt,debt
children,,,
5,0,9,0.0%
0,1063,14091,7.54%
3,27,330,8.18%
1,445,4855,9.17%
2,202,2128,9.49%
4,4,41,9.76%


### Вывод

Самая низкая вероятность задолженности у людей без детей. Наивысшая - у семей с четырьмя детьми

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [38]:
# поскольку мы до этого удаляли из основных данных текстовые значения family_status, тут необходимо их вернуть

df_with_family_status = df.merge(family_status_data, on='family_status_id', how='left')

# построим сводную таблицу для ответа на вопрос (по фрейму со значениями из словаря)
data_pivot = df_with_family_status.pivot_table(index=['family_status_id'], values=['debt'], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot


,sum,count,make_proportion
,debt,debt,debt
family_status_id,,,
2,63,959,6.57%
3,85,1195,7.11%
0,931,12339,7.55%
1,388,4151,9.35%
4,274,2810,9.75%


### Вывод

Вдовец/вдова - категория с самым низким процентом просрочки. Возможно, это связано с тем, что кредит берется осознанно.

Неженатые и состоящие в гражданском браке клиенты, имеют самую большую просрочку.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['total_income_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot

,sum,count,make_proportion
,debt,debt,debt
total_income_category,,,
D,21,350,6.0%
B,356,5041,7.06%
A,2,25,8.0%
C,1360,16016,8.49%
E,2,22,9.09%


### Вывод

Тут видно, что у клиентов с более высоким доходом имеется наибольшая просрочка, в то время как "средний класс" имеет меньшую просрочку по оплатам. Клиенты с диаметрально противоположными доходами в кредитах не нуждаются.

### Как разные цели кредита влияют на его возврат в срок?

In [40]:
# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['purpose_category'], values=['debt'], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot

,sum,count,make_proportion
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,782,10811,7.23%
проведение свадьбы,186,2324,8.0%
получение образования,370,4013,9.22%
операции с автомобилем,403,4306,9.36%


### Вывод

Выгоднее всего давать кредиты на недвижимость, а автомобиль и образование - в зоне риска. Хоть абсолютная разница не такая большая, относительно друг друга она составляет порядка 22%, что уже не мало

## Общий вывод:

Самый идеальный заемщик - это человек без детей, состоящий или состоявший в официальных отношениях, имеющий уровень дохода выше среднего и берущий деньги на недвижимость.

"Разброс" просроченных выплат по каждой из категорий примерно одинаковый. В среднем, это диапазоны от 7 до 9 процентов. В остальном разница в абсолютных цифрах не такая большая. Однако в относительных разница составляет в среднем 2 процента, что означает разницу в вероятности просрочки платежа на 20% между более

Детально по категориям:

Влияние наличия детей на вероятность просрочки платежа
По наличию детей
* 5 - 0.0% (незначительное кличество, учитывать не будем)
* 0 - 7.54%
* 3 - 8.18%
* 1 - 9.16%
* 2 - 9.49%
* 4 - 9.76%

**Абсолютная разница между минимальным и максимальным значением - 2.22%

**Относительная разница между минимальным и максимальным значением - 24%


Влияние семейного положения на вероятность просрочки платежа
* По семейному положению
* вдовец / вдова - 6.57%
* в разводе - 7.11%
* женат / замужем - 7.54%
* гражданский брак - 9.32%
* Не женат / не замужем - 9.75%

**Абсолютная разница между минимальным и максимальным значением - 3.18%

**Относительная разница между минимальным и максимальным значением - 33%


Влияние уровня дохода на просрочку платежа

По уровню дохода
* D (30001–50000) - 6.0%
* B (200001–1000000) - 7.06%
* A (1000001)	- 8.0%
* C (50001–200000) - 8.48%
* E (До 30000) - 9.09%

**Абсолютная разница между минимальным и максимальным значением - 3.09%

**Относительная разница между минимальным и максимальным значением - 34%


Влияние цели кредита на вероятность возврата кредита в срок

По цели кредита
* недвижимость - 7.23%
* свадьба - 8.0%
* образование - 9.22%
* автомобиль - 9.36%

**Абсолютная разница между минимальным и максимальным значением - 2.13%

**Относительная разница между минимальным и максимальным значением - 23%


При этом, нужно отметить, что размер выборки уровня дохода для значений "до 30000" и "больше 1000000" мал. Из этого можно сделать вывод, что люди со сверхдоходами не так часто нуждаются в кредитах, а также и то, что люди с небольшим доходом также опасаются их брать.Однако же, это означает, что итоговые данные по этим выборкам могут быть статистически неточными.

Поэтому, уточним более конкретный диапазон доходов среднего класса, исключив "сверховысокий" и "сверхнизкий" доходы
* D (30001–50000) - 6.0%
* B (200001–1000000) - 7.06%
* C (50001–200000) - 8.48%

**Абсолютная разница между минимальным и максимальным значением - 2.48%

**Относительная разница между минимальным и максимальным значением - 29%


Таким образом, несмотря на схожий в процентном отношении разброс значений, наибольшее влияние на вероятность просрочки показывает именно семейное положение. Во вторую очередь, следует обращать внимание на уровень дохода. Наименьшее влияние оказывают цели кредита и наличие детей.